# Setup

In [21]:
import os
import openai
import time
 
openai.api_key = os.environ["OPENAI_API_KEY"]

def llm_legacy(prompt, stop=["\n"]):
    # response = openai.Completion.create(
    response = openai.completions.create(
      model="text-davinci-002",
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response.choices[0].text ##
    # return response["choices"][0]["text"]

def llm(prompt, stop=["\n"]):
    response = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=prompt,
      temperature=0,
      max_tokens=200,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response.choices[0].message.content

In [22]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

Example for messages:
```
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
```

Full documentation: https://platform.openai.com/docs/api-reference/chat

In [23]:
def append_message(messages, role, content):
    messages.append({
        "role": role,
        "content": content
    })
    return messages

def get_one_message(messages, role="system"):
    final_message = ""
    for message in messages:
        if message["role"] != role:
            final_message += message["content"] + "\n"
    return final_message

# ReAct

In [24]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples

webthink_critique_examples = prompt_dict['webthink_critique_examples'] ##
# instruction_critique = """You are a helpful assistant. You are assisting another agent who's trying to solve a question answering task. You will find Thought, Action steps. Thought can reason about the current situation, and Action can be three types:
# (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
# (2) Lookup[keyword], which returns the next sentence containing the keyword in the current passage.
# (3) Finish[answer], which returns the answer and finishes the task.
# Ensure that the thoughts and actions are directed towards the most efficient and direct path to answer the given question, avoiding unnecessary steps or exhaustive searches.
# Always include a Critique in your response. Suggest a new Thought, Action pair in the Critique only if corrections are necessary.
# Be sure to not include indices or Observation when suggesting a new Thought, Action pair.
# Here are some examples.
# """ ##
# instruction_critique = """You are a helpful assistant. You are assisting another agent who's trying to solve a question answering task. Ensure that the thoughts and actions are directed towards the most correct and efficient path to answer the given question. You will find Thought, Action steps. Thought can reason about the current situation, and Action can be three types:
# # (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
# # (2) Lookup[keyword], which returns the next sentence containing the keyword in the current passage.
# # (3) Finish[answer], which returns the answer and finishes the task.
# Always include a Critique in your response. Suggest a new Thought, Action pair in the Critique only if corrections are necessary.
# Be sure to not include indices of the type 1, 2, 3 after Thought or Action. Do not include new Observation.
# Here are some examples.
# """ ##
instruction_critique = """You are a helpful assistant. You are assisting another agent who's trying to solve a question answering task. Ensure that the thoughts and actions are directed towards the most correct and efficient path to answer the given question. You will find Thought, Action steps. Thought can reason about the current situation, and Action can be three types:
# (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
# (2) Lookup[keyword], which returns the next sentence containing the keyword in the current passage.
# (3) Finish[answer], which returns the answer and finishes the task.
Always include a Critique in your response. Suggest a new Thought, Action pair in the Critique only if corrections are necessary.
Here are some examples.
""" ##
webthink_prompt_critique = instruction_critique + webthink_critique_examples ##

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
# def webthink(messages, idx=None, to_print=True):
    # print("START messages", messages) ##
    question = env.reset(idx=idx)
    # question = "What does the goddess associated with the goddess frigg consists of what tales?" ##
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    # messages = append_message(messages, "user", question + "\n") ##
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        # time.sleep(5) ##
        print("i", i) ##
        n_calls += 1
        # messages_temp = messages.copy() ##
        # if i == 1: ##
        #     messages_temp[-1]['content'] += f"Thought {i}:" ##
        # else: ##
        #     messages_temp = append_message(messages_temp, "user", f"Thought {i}:") ##
        # print("ONGOING messages", messages) ##
        thought_action = llm_legacy(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        # thought_action = llm(messages_temp, stop=[f"\nObservation {i}:"])
        # print("thought_action", thought_action) ##
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
            # if i != 1: ##
            #     thought = thought.split(f"Thought {i}: ")[1] ##
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            # messages_temp[-1]['content'] = f"Thought {i}: {thought}\nAction {i}:" ##
            action = llm_legacy(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
            # time.sleep(5) ##
            # action = llm(messages_temp, stop=[f"\n"]).strip() ##

        # TODO: critique the (Thought i, Action i) pair
        # messages_critique = append_message([], "system", webthink_prompt_critique) ##
        # messages_critique = append_message(messages_critique, "user", get_one_message(messages) + f"Thought {i}: {thought}\nAction {i}: {action}") ##
        # critique = llm(messages_critique, stop=None).strip() ## TODO: non fa la critica e si limita a ripetere il flow del ragionamento. Se fa correzzioni le copia da quelle proposte dall'agente principale
        # if f"Thought:" in critique: ##
        #     thought, action = critique.split(f"\nAction: ") ##
        #     thought = thought.split(f"Thought: ")[1] ##
        #     thought = thought.replace("Thought:", f"Thought {i}:") ##
        #     action = action.replace("Action:", f"Action {i}:") ##
        # TODO: critique the (Thought i, Action i) pair

        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        # messages = append_message(messages, "assistant", step_str) ##
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt}) ##
    return r, info

# import random
# import time

# random.seed(42)

# idxs = list(range(7405))
# random.Random(233).shuffle(idxs)

# rs = []
# infos = []
# old_time = time.time()

# for i in idxs[:1]:
#     messages = append_message([], "system", webthink_prompt) ##
#     r, info = webthink(messages, i, to_print=True)
#     rs.append(info['em'])
#     infos.append(info)
#     print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
#     print('-----------')
#     print()

In [25]:
import random
import time

# random.seed(42) ##

idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()

for i in idxs[:1]:
    # messages = append_message([], "system", webthink_prompt) ##
    # r, info = webthink(messages, i, to_print=True) ##
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

3687 Question: What movie did actress Irene Jacob complete before the American action crime thriller film directed by Stuart Baird?
i 1
Thought 1: I need to search Irene Jacob and find the movie she completed before the American action crime thriller film directed by Stuart Baird.
Action 1: Search[Irene Jacob]
Observation 1: Irène Marie Jacob (born 15 July 1966) is a French-Swiss actress known for her work with Polish film director Krzysztof Kieślowski.[2] She won the 1991 Cannes Film Festival Award for Best Actress for the Kieślowski film The Double Life of Veronique, and was nominated for the BAFTA Award for Best Actress in a Leading Role for her 1994 film Three Colours: Red. Her other film appearances include The Secret Garden (1993), Beyond the Clouds (1995), U.S. Marshals (1998), and Eternity (2016).. Irène Jacob was born in Suresnes, Hauts-de-Seine, a western suburb of Paris.[3] The youngest child with three older brothers, she was raised in a highly educated and intellectual fam